# AUC

In [ ]:
import my_datasets as md
import my_models as mm
import my_uncertainty as mu
import torch
from torch import nn
import scipy.stats as stats
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import roc_auc_score

with open('results.pkl', 'rb') as f: 
    r = pickle.load(f)

plt.rcParams["figure.figsize"] = [20, 4]
UNCER_QUANT = [mu.BLOODQuant().name+"_mean", mu.BLOODQuant().name+"_last", mu.LeastConfidentQuant().name, mu.EntropyQuant().name, mu.MCDropoutQuant().name, mu.GradQuant().name, mu.EnsambleQuant().name, mu.TemperatureScalingQuant().name, mu.MahalanobisQuant().name, "repr_change_mean", "repr_change_last"]
MODELS = ["RoBERTa"] 
NUM_SEEDS = 5

def variation(uncer, mode="_last"):
    if mode == "_last":
        return uncer[-1,:]
    elif mode == "_mean":
        return uncer.mean(axis=0)
    
for data_name, run in r.items():
    for model_name in MODELS:
        for uncer_name in UNCER_QUANT:
            BIO = False
            for seed in range(NUM_SEEDS):
                if (uncer_name.startswith(mu.BLOODQuant().name) or uncer_name.startswith("repr_change")) and not BIO:
                    BIO = True
                    mode = uncer_name[-5:]
                    uncer_name = uncer_name[:-5]
                elif not (uncer_name.startswith(mu.BLOODQuant().name) or uncer_name.startswith("repr_change")):
                    mode = ""
                uncer_id = run[model_name]["fine-tuned"][seed][uncer_name]["id"]
                uncer_id = variation(uncer_id, mode=mode) if uncer_name in ["BLOOD", "repr_change"] else uncer_id
                uncer_id = -uncer_id if uncer_name == "repr_change" else uncer_id
                uncer_ood = run[model_name]["fine-tuned"][seed][uncer_name]["ood"]
                uncer_ood = variation(uncer_ood, mode=mode) if uncer_name in ["BLOOD", "repr_change"] else uncer_ood
                uncer_ood = -uncer_ood if uncer_name == "repr_change" else uncer_ood
                DATA = [0 for _ in range(len(uncer_id))] + [1 for _ in range(len(uncer_ood))]
                aucs.append(roc_auc_score(DATA, uncer_id.tolist()+uncer_ood.tolist()))
            fig.show()
            aucs = np.array(aucs)
            print(f"{data_name} - {model_name} [{uncer_name+mode}]: AUC = {round(aucs.mean()*100, 2):.2f} $\pm$ {round(aucs.std()*100, 2):.2f}")
        print()

In [ ]:
import my_datasets as md
import my_models as mm
import my_uncertainty as mu
import torch
from torch import nn
import scipy.stats as stats
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import roc_curve, roc_auc_score

with open('results-shifted-sst.pkl', 'rb') as f: 
    r = pickle.load(f)
    
plt.rcParams["figure.figsize"] = [20, 4]
UNCER_QUANT = [mu.BLOODQuant().name+"_mean", mu.BLOODQuant().name+"_last", mu.LeastConfidentQuant().name, mu.EntropyQuant().name, mu.MCDropoutQuant().name, mu.GradQuant().name, mu.EnsambleQuant().name, mu.TemperatureScalingQuant().name, mu.MahalanobisQuant().name, "repr_change_mean", "repr_change_last"]
MODELS = ["RoBERTa", "ELECTRA"] 
NUM_SEEDS = 5

def variation(uncer, mode="last"):
    if mode == "last":
        return uncer[-1,:]
    elif mode == "mean":
        return uncer.mean(axis=0)
    
for data_name, run in r.items():
    for model_name in MODELS:
        for uncer_name in UNCER_QUANT:
            for seed in range(NUM_SEEDS):               
                uncer_id = run[model_name]["fine-tuned"][seed][uncer_name]["id"]
                uncer_id = variation(uncer_id) if uncer_name in ["BLOOD", "repr_change"] else uncer_id
                uncer_id = -uncer_id if uncer_name == "repr_change" else uncer_id
                uncer_ood = run[model_name]["fine-tuned"][seed][uncer_name]["shifted"]
                uncer_ood = variation(uncer_ood) if uncer_name in ["BLOOD", "repr_change"] else uncer_ood
                uncer_ood = -uncer_ood if uncer_name == "repr_change" else uncer_ood
                
                DATA = [0 for _ in range(len(uncer_id))] + [1 for _ in range(len(uncer_ood))]
                fpr, tpr, _ = roc_curve(DATA, uncer_id.tolist()+uncer_ood.tolist())
                aucs.append(roc_auc_score(DATA, uncer_id.tolist()+uncer_ood.tolist()))
            aucs = np.array(aucs)
            print(f"{data_name} - {model_name} [{uncer_name}]: AUC = {round(aucs.mean()*100, 2):.2f} $\pm$ {round(aucs.std()*100, 2):.2f}"))
        print()

# AUPR

In [ ]:
import my_datasets as md
import my_models as mm
import my_uncertainty as mu
import torch
from torch import nn
import scipy.stats as stats
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import auc, precision_recall_curve

with open('results.pkl', 'rb') as f: 
    r = pickle.load(f)
    
plt.rcParams["figure.figsize"] = [20, 4]
UNCER_QUANT = [mu.BLOODQuant().name+"_mean", mu.BLOODQuant().name+"_last", mu.LeastConfidentQuant().name, mu.EntropyQuant().name, mu.MCDropoutQuant().name, mu.GradQuant().name, mu.EnsambleQuant().name, mu.TemperatureScalingQuant().name, mu.MahalanobisQuant().name]
MODELS = ["ELECTRA"] 
NUM_SEEDS = 5

def variation(uncer, mode="_last"):
    if mode == "_last":
        return uncer[-1,:]
    elif mode == "_mean":
        return uncer.mean(axis=0)
    
for data_name, run in r.items():
    for model_name in MODELS:
        for uncer_name in UNCER_QUANT:
            BIO = False
            fig, axs = plt.subplots(1, NUM_SEEDS)
            if NUM_SEEDS == 1:
                axs = [axs]
            aucs = []
            for seed in range(NUM_SEEDS):
                uncer_id = run[model_name]["fine-tuned"][seed][uncer_name]["id"]
                uncer_id = variation(uncer_id, mode=mode) if uncer_name in ["BLOOD", "repr_change"] else uncer_id
                uncer_id = -uncer_id if uncer_name == "repr_change" else uncer_id
                uncer_ood = run[model_name]["fine-tuned"][seed][uncer_name]["ood"]
                uncer_ood = variation(uncer_ood, mode=mode) if uncer_name in ["BLOOD", "repr_change"] else uncer_ood
                uncer_ood = -uncer_ood if uncer_name == "repr_change" else uncer_ood
                DATA = [1 for _ in range(len(uncer_id))] + [0 for _ in range(len(uncer_ood))]
                uncer_id = -uncer_id
                uncer_ood = -uncer_ood
                precision, recall, _ = precision_recall_curve(DATA, uncer_id.tolist()+uncer_ood.tolist())
                aucs.append(auc(recall, precision))
            aucs = np.array(aucs)
            print(f"{data_name} - {model_name} [{uncer_name+mode}]: AUPR = {round(aucs.mean()*100, 2):.2f} $\pm$ {round(aucs.std()*100, 2):.2f}")
        print()

In [ ]:
import my_datasets as md
import my_models as mm
import my_uncertainty as mu
import torch
from torch import nn
import scipy.stats as stats
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import roc_curve, roc_auc_score

with open('results-shifted-sst.pkl', 'rb') as f: 
    r = pickle.load(f)
    
plt.rcParams["figure.figsize"] = [20, 4]
UNCER_QUANT = [mu.BLOODQuant().name+"_mean", mu.BLOODQuant().name+"_last", mu.LeastConfidentQuant().name, mu.EntropyQuant().name, mu.MCDropoutQuant().name, mu.GradQuant().name, mu.EnsambleQuant().name, mu.TemperatureScalingQuant().name, mu.MahalanobisQuant().name]
MODELS = ["RoBERTa", "ELECTRA"]
NUM_SEEDS = 5

def variation(uncer, mode="last"):
    if mode == "last":
        return uncer[-1,:]
    elif mode == "mean":
        return uncer.mean(axis=0)
    
for data_name, run in r.items():
    for model_name in MODELS:
        for uncer_name in UNCER_QUANT:
            if NUM_SEEDS == 1:
                axs = [axs]
            aucs = []
            for seed in range(NUM_SEEDS):               
                uncer_id = run[model_name]["fine-tuned"][seed][uncer_name]["id"]
                uncer_id = variation(uncer_id) if uncer_name in ["BLOOD", "repr_change"] else uncer_id
                uncer_id = -uncer_id if uncer_name == "repr_change" else uncer_id
                uncer_ood = run[model_name]["fine-tuned"][seed][uncer_name]["shifted"]
                uncer_ood = variation(uncer_ood) if uncer_name in ["BLOOD", "repr_change"] else uncer_ood
                uncer_ood = -uncer_ood if uncer_name = "repr_change" else uncer_ood
                
                DATA = [1 for _ in range(len(uncer_id))] + [0 for _ in range(len(uncer_ood))]
                uncer_id = -uncer_id
                uncer_ood = -uncer_ood
                precision, recall, _ = precision_recall_curve(DATA, uncer_id.tolist()+uncer_ood.tolist())
                aucs.append(auc(recall, precision))
            aucs = np.array(aucs)
            print(f"{data_name} - {model_name} [{uncer_name}]: AUPR = {round(aucs.mean()*100, 2):.2f} $\pm$ {round(aucs.std()*100, 2):.2f}")
        print()

# FPR@95TPR

In [5]:
import numpy as np
import torch

def fpr95(uncers_id, uncers_ood):
    th = np.percentile(uncers_id, 95)
    if torch.is_tensor(uncers_ood):
        uncers_ood = uncers_ood.cpu().detach().numpy()
    return (uncers_ood < th).mean()

In [ ]:
import my_datasets as md
import my_models as mm
import my_uncertainty as mu
import torch
from torch import nn
import scipy.stats as stats
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

with open('results.pkl', 'rb') as f: 
    r = pickle.load(f)
    
plt.rcParams["figure.figsize"] = [20, 4]
UNCER_QUANT = [mu.BLOODQuant().name+"_mean", mu.BLOODQuant().name+"_last", mu.LeastConfidentQuant().name, mu.EntropyQuant().name, mu.MCDropoutQuant().name, mu.GradQuant().name, mu.EnsambleQuant().name, mu.TemperatureScalingQuant().name, mu.MahalanobisQuant().name]
MODELS = ["ELECTRA"] 
NUM_SEEDS = 5

def variation(uncer, mode="_last"):
    if mode == "_last":
        return uncer[-1,:]
    elif mode == "_mean":
        return uncer.mean(axis=0)
    
for data_name, run in r.items():
    for model_name in MODELS:
        for uncer_name in UNCER_QUANT:
            BIO = False
            fig, axs = plt.subplots(1, NUM_SEEDS)
            if NUM_SEEDS == 1:
                axs = [axs]
            aucs = []
            for seed in range(NUM_SEEDS):
                uncer_id = run[model_name]["fine-tuned"][seed][uncer_name]["id"]
                uncer_id = variation(uncer_id, mode=mode) if uncer_name in ["BLOOD", "repr_change"] else uncer_id
                uncer_id = -uncer_id if uncer_name == "repr_change" else uncer_id
                uncer_ood = run[model_name]["fine-tuned"][seed][uncer_name]["ood"]
                uncer_ood = variation(uncer_ood, mode=mode) if uncer_name in ["BLOOD", "repr_change"] else uncer_ood
                uncer_ood = -uncer_ood if uncer_name == "repr_change" else uncer_ood
                aucs.append(fpr95(uncer_id, uncer_ood))
            aucs = np.array(aucs)
            print(f"{data_name} - {model_name} [{uncer_name+mode}]: FPR = {round(aucs.mean()*100, 2):.2f} $\pm$ {round(aucs.std()*100, 2):.2f}")
        print()

In [ ]:
import my_datasets as md
import my_models as mm
import my_uncertainty as mu
import torch
from torch import nn
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import roc_curve, roc_auc_score

with open('results-shifted-sst.pkl', 'rb') as f: 
    r = pickle.load(f)
    
plt.rcParams["figure.figsize"] = [20, 4]
UNCER_QUANT = [mu.BLOODQuant().name+"_mean", mu.BLOODQuant().name+"_last", mu.BLOODQuant().name, mu.EntropyQuant().name, mu.MCDropoutQuant().name, mu.GradQuant().name, mu.EnsambleQuant().name, mu.TemperatureScalingQuant().name, mu.MahalanobisQuant().name]
MODELS = ["RoBERTa", "ELECTRA"]
NUM_SEEDS = 5

def variation(uncer, mode="last"):
    if mode == "last":
        return uncer[-1,:]
    elif mode == "mean":
        return uncer.mean(axis=0)
    
for data_name, run in r.items():
    for model_name in MODELS:
        for uncer_name in UNCER_QUANT:
            if NUM_SEEDS == 1:
                axs = [axs]
            aucs = []
            for seed in range(NUM_SEEDS):               
                uncer_id = run[model_name]["fine-tuned"][seed][uncer_name]["id"]
                uncer_id = variation(uncer_id) if uncer_name in ["BLOOD", "repr_change"] else uncer_id
                uncer_id = -uncer_id if uncer_name == "repr_change" else uncer_id
                uncer_ood = run[model_name]["fine-tuned"][seed][uncer_name]["shifted"]
                uncer_ood = variation(uncer_ood) if uncer_name in ["BLOOD", "repr_change"] else uncer_ood
                uncer_ood = -uncer_ood if uncer_name == "repr_change" else uncer_ood
                
                aucs.append(fpr95(uncer_id, uncer_ood))
                
            aucs = np.array(aucs)
            print(f"{data_name} - {model_name} [{uncer_name}]: AUPR = {round(aucs.mean()*100, 2):.2f} $\pm$ {round(aucs.std()*100, 2):.2f}")